In [1]:
%run web_scrape_func_def.ipynb

> file management func:

    get_content(directory)
    get_last_update(file_path)

> Updating lists func:

    get_participants(event_link)

> fighters_stats_scrape func:

    get_xml(url, data_type='xml', headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)

> upcoming_web_scrape func:

    get_upcoming_event_url(xml)
    get_upcoming_event_data(upcoming_event_url)

> fighters_stats_full_update func:

    get_fighters_links(page_url)

> draftking_odds func:

    get_table_odds(table)
    get_odds(data, fighter, opponent)


# Main( )

In [2]:
data_loc = '../data/'

In [3]:
events = pd.read_csv(data_loc + 'update_list_events.csv')
events.tail(3)

,date,event_url
646,1995-04-07,http://ufcstats.com/event-details/b60391da771d...
647,1994-12-16,http://ufcstats.com/event-details/1a49e0670dfa...
648,1994-09-09,http://ufcstats.com/event-details/a6a9ab5a824e...


### get event name - staging

In [8]:
url = events.loc[0,'event_url']
print(url)
get_event_data(event_link=url)

http://ufcstats.com/event-details/b6c6d1731ff00eeb


[['win',
  'Amanda Nunes',
  'Irene Aldana',
  '0',
  '0',
  '142',
  '41',
  '6',
  '0',
  '0',
  '0',
  "Women's Bantamweight",
  'U-DEC',
  '',
  '5',
  '5:00',
  '06-10-2023',
  'http://ufcstats.com/event-details/b6c6d1731ff00eeb'],
 ['win',
  'Charles Oliveira',
  'Beneil Dariush',
  '1',
  '0',
  '26',
  '12',
  '0',
  '0',
  '0',
  '0',
  'Lightweight',
  'KO/TKO',
  'Punches',
  '1',
  '4:10',
  '06-10-2023',
  'http://ufcstats.com/event-details/b6c6d1731ff00eeb'],
 ['win',
  'Mike Malott',
  'Adam Fugitt',
  '1',
  '0',
  '19',
  '9',
  '2',
  '0',
  '1',
  '0',
  'Welterweight',
  'SUB',
  'Guillotine Choke',
  '2',
  '1:06',
  '06-10-2023',
  'http://ufcstats.com/event-details/b6c6d1731ff00eeb'],
 ['win',
  'Dan Ige',
  'Nate Landwehr',
  '1',
  '0',
  '88',
  '74',
  '0',
  '0',
  '0',
  '0',
  'Featherweight',
  'U-DEC',
  '',
  '3',
  '5:00',
  '06-10-2023',
  'http://ufcstats.com/event-details/b6c6d1731ff00eeb'],
 ['win',
  'Marc-Andre Barriault',
  'Eryk Anders',
  '1',

In [11]:
xml = get_xml(url=url)
xml

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html class="no-js ie8 lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js"> <!--<![endif]-->
<head>
<meta charset="utf-8">
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<title>
    Stats | UFC
  </title>
<meta content="" name="description">
<meta content="" name="viewport">
<link href="/blocks/main.css?ver=440510" rel="stylesheet">
<script src="/js/vendor/modernizr-2.6.2.min.js"/>
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
    m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
    })(window,document,'script','//www.google-analytics.com/analytics.js','ga');

    ga('create', 'UA-2855164-1', 

#### Extracting data from website to data frame:

In [4]:
data = []
column_names = ['result','fighter','opponent',
                'f_kd','o_kd','f_str','o_str','f_td','o_td','f_sub','o_sub',
                'weight_class','method','type','round','time','date','event_url'
               ]

for url in tqdm(events.event_url):
    data.extend(get_event_data(event_link=url))
    
df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.40it/s]


,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
30,win,Rodrigo Nascimento,Ilir Latifi,0,0,68,49,0,2,0,0,Heavyweight,S-DEC,,3,5:00,05-20-2023,http://ufcstats.com/event-details/6f812143641c...
31,win,Chase Hooper,Nick Fiore,0,0,149,51,1,4,1,0,Lightweight,U-DEC,,3,5:00,05-20-2023,http://ufcstats.com/event-details/6f812143641c...
32,win,Natalia Silva,Victoria Leonardo,1,0,20,7,0,0,0,0,Women's Flyweight,KO/TKO,Punch,1,2:58,05-20-2023,http://ufcstats.com/event-details/6f812143641c...


#### Merge data with existing file: 

In [5]:
org = pd.read_csv(data_loc + 'ufc_matches_stats.csv')
org.tail(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
6492,win,Norma Dumont,Karol Rosa,0,1,35,33,1,0,0,0,Women's Featherweight,U-DEC,NaN,3,5:00,04-22-2023,http://ufcstats.com/event-details/6fb1ba67bef4...
6493,win,Mohammed Usman,Junior Tafa,0,0,7,22,2,0,0,0,Heavyweight,U-DEC,NaN,3,5:00,04-22-2023,http://ufcstats.com/event-details/6fb1ba67bef4...
6494,win,William Gomis,Francis Marshall,0,0,27,15,0,2,0,2,Featherweight,S-DEC,NaN,3,5:00,04-22-2023,http://ufcstats.com/event-details/6fb1ba67bef4...


In [6]:
data = pd.concat([org, df]).drop_duplicates(subset=['fighter','opponent','date']).reset_index(drop=True)
data.tail(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
6514,win,Muhammad Naimov,Jamie Mullarkey,1,0,28,39,0,3,0,0,Lightweight,KO/TKO,Punch,2,2:59,06-03-2023,http://ufcstats.com/event-details/7abe471b6172...
6515,win,Elise Reed,Jinh Yu Frey,0,0,53,28,0,3,0,0,Women's Strawweight,U-DEC,,3,5:00,06-03-2023,http://ufcstats.com/event-details/7abe471b6172...
6516,win,Da'Mon Blackshear,Luan Lacerda,0,0,66,28,1,1,0,1,Bantamweight,KO/TKO,Punches,2,3:54,06-03-2023,http://ufcstats.com/event-details/7abe471b6172...


#### Saving file:

In [7]:
data.to_csv(data_loc + 'ufc_matches_stats.csv', index=0)
data.tail(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
6514,win,Muhammad Naimov,Jamie Mullarkey,1,0,28,39,0,3,0,0,Lightweight,KO/TKO,Punch,2,2:59,06-03-2023,http://ufcstats.com/event-details/7abe471b6172...
6515,win,Elise Reed,Jinh Yu Frey,0,0,53,28,0,3,0,0,Women's Strawweight,U-DEC,,3,5:00,06-03-2023,http://ufcstats.com/event-details/7abe471b6172...
6516,win,Da'Mon Blackshear,Luan Lacerda,0,0,66,28,1,1,0,1,Bantamweight,KO/TKO,Punches,2,3:54,06-03-2023,http://ufcstats.com/event-details/7abe471b6172...
